# 1. Define the Problem

From https://machinelearningmastery.com/process-for-working-through-machine-learning-problems/

I like to use a three step process to define the problem. I like to move quickly and I use this mini process to see the problem from a few different perspectives very quickly:

* Step 1: What is the problem? Describe the problem informally and formally and list assumptions and similar problems.
* Step 2: Why does the problem need to be solved? List your motivation for solving the problem, the benefits a solution provides and how the solution will be used.
* Step 3: How would I solve the problem? Describe how the problem would be solved manually to flush domain knowledge.


## 1.1 What is the problem? 
Given a set of metrics organised by type and country, we need to predict a subset of these metrics for one year ahead and five years ahead.
The data is quite sparse and very few entries have values for the complete time range of 1972-2007. It also appears that metric names may not be consistent between countries. The dataset will require a lot of cleaning before we can use it.

## 1.2 Why does the problem need to be solved?

### 1.2.1 From a global perspective
* The United Nations millenium development goals are a way of measuring the level of development of countries around the world in terms of important metrics such as poverty levels and female empowerment. It would be very useful to be able to predict when a given country might reach the levels the UN has set as targets.

### 1.2.2 From a study group perspective
* This dataset gives us a chance to work with time-series data (which we have not addressed as a group before)
* The dataset is quite sparse with many missing values and possible inconsistent fields. A successfull result with this will give us badly needed experience with data cleaning.

## 1.3 How would I solve the problem?
TBD

# 1.4 Current status 24/4/2019
There are two files in the dataset
* training data which gives the values of particular metrics (AKA series) for a given country for the years 1972-2007 (many have years missing)
* submission data which gives us the row IDs we need to predict one year (2008) and five years (2012) into the future

What we have learned so far
* The series name and series code are perfectly correlated, we can drop the series name with no loss of data
* The series codes may not be consistent. Some may include a country code and others not. The names of the series also appear to vary between countries. It's likely we'll need to do some work to come up with a consistent set of series code across the dataset.
* The first column in the training and submission sets is a row ID that we can use to join the two datasets
* If we consider only the joined dataset then we have a much higher proportion of year data for series than the dataset as a whole. We also only need to predict values for 737 series/country combinations out of the 195402 present in the training set.
* The 737 desired values may not be enough to train a model so we can't necessarily discard 194665 (195402-737) rows
* We can problem frame this as a regression problem. Given N years of data for a metric, predict next year

Next steps
* Convert year values from columns to rows: Convert 2D table to 3D dataset. This will make it easier to drop empty values
* Prove correlation between series name and series code
* separate series code and determine whether that correlates with country or is orthogonal
* Work out what data can be dropped
* try framing as regression problem: how many years do we need?
* are metrics independent can we train on a metric/country basis or do we need some method for training per metric (aggregate across countries)